In [2]:

import numpy as np 
import pandas as pd 


This notebook has been inspired by the short FREE course offered by  'Career Foundry' (https://careerfoundry.com); in this course the tutor uses google sheets to walk the prospective students through a data analysis mini project. Here I will replicate this 'walk' using pandas. 

What is 'pandas'?
"pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series..." (source:https://en.wikipedia.org/wiki/Pandas_(software)); in short, for the Python affecionados, pandas does everything that .xls can do, but better :)

pandas is a Python library( python library (rough definition): collection of ready made functions ); 

In google sheets the software and the working enverionment,i.e the worksheet are launched together, in order to use pandas functionality pandas has to be imported into your work space; this may feel unfamiliar if coming from a .xls environment,howver it is very simple to import libraries into a notebook:

In [ ]:
# hash is used to add quick information about what you are going to do. 
# import libraries 
import pandas as pd # I am telling the system to retrieve the pandas library, and that from now on when I need to use it I will call it pd 
import numpy as np # numpy is another library , conventionally pandas and numpy are imported at the same time, I am keeping with the convention here although I will not need numpy here. 

In [3]:
# import sql data
import sqlite3
from sqlite3 import Error

In [4]:
import pandas as pd
import sqlite3

con = sqlite3.connect('../input/new-york-city-taxi-trips-2019/2019/2019-01.sqlite')

df = pd.read_sql('SELECT * FROM tripdata', con)


In [10]:
(round(df['fare_amount'].describe().T),2)

(count    7667792.0
 mean          12.0
 std          262.0
 min         -362.0
 25%            6.0
 50%            8.0
 75%           14.0
 max       623260.0
 Name: fare_amount, dtype: float64,
 2)

In [9]:
df[df['fare_amount']<0]

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
663,2.0,2019-01-01 00:32:56.000000,2019-01-01 00:33:35.000000,2.0,0.10,1.0,N,148.0,148.0,3.0,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,NaN
2402,2.0,2019-01-01 00:14:18.000000,2019-01-01 00:40:55.000000,1.0,4.13,1.0,N,170.0,238.0,4.0,-19.0,-0.5,-0.5,0.0,0.0,-0.3,-20.3,NaN
2541,2.0,2019-01-01 00:31:27.000000,2019-01-01 00:42:39.000000,1.0,1.35,1.0,N,162.0,234.0,4.0,-8.5,-0.5,-0.5,0.0,0.0,-0.3,-9.8,NaN
2544,2.0,2019-01-01 00:45:57.000000,2019-01-01 00:46:07.000000,1.0,0.00,1.0,N,234.0,234.0,4.0,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,NaN
2547,2.0,2019-01-01 00:48:35.000000,2019-01-01 00:49:59.000000,1.0,0.16,1.0,N,234.0,234.0,4.0,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7665612,2.0,2019-01-31 23:24:31.000000,2019-01-31 23:25:52.000000,1.0,0.34,1.0,N,161.0,237.0,3.0,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,0.0
7666343,2.0,2019-01-31 23:36:37.000000,2019-01-31 23:37:44.000000,1.0,0.17,1.0,N,68.0,158.0,3.0,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,0.0
7666452,2.0,2019-01-31 23:08:30.000000,2019-01-31 23:19:27.000000,2.0,0.58,1.0,N,162.0,229.0,3.0,-8.0,-0.5,-0.5,0.0,0.0,-0.3,-9.3,0.0
7666601,2.0,2019-01-31 23:42:48.000000,2019-01-31 23:48:23.000000,1.0,0.68,1.0,N,211.0,211.0,4.0,-5.5,-0.5,-0.5,0.0,0.0,-0.3,-6.8,0.0


In [13]:
df['fare_amount'].idxmax()

2499600